In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import keras
from music_generator.basic.random import generate_dataset
from music_generator.basic.signalproc import SamplingInfo
from music_generator.musical.timing import Tempo
from music_generator.musical.scales import GenericScale
from music_generator.analysis.play import play_mono_as_stereo, play_array
from music_generator.basic.signalproc import mix_at
from music_generator.analysis.preprocessing import decoder_predict
from music_generator.analysis.preprocessing import apply_fourier_on_input
from music_generator.basic.utils import parallel_apply_along_axis
from music_generator.analysis.preprocessing import create_training_data_set, model_predict, apply_fourier_on_dataset

from scipy.io.wavfile import read
from scipy.fftpack import rfft, irfft

import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
%matplotlib inline

/home/marcelra/Pycharmprojects/music-generator/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def read_file(filename):
    x = read(filename)
    return x[1][:, 0] / 2**15

In [4]:
bt_0 = read_file("../data/Verplichte Kots_v9.wav")
tot_0 = read_file("../data/Verplichte Kots_v9.wav")

In [5]:
# play_array(bt_0, range_secs=[20, 24], sample_rate=48000)
# play_array(tot_0, range_secs=[20, 24], sample_rate=48000)

In [6]:
rel_offset = int(3.135*44100)
x = bt_0[:10*44100 - rel_offset] - tot_0[rel_offset:10*44100]
#play_array(x, range_secs=[0,4])

## Make selection in song

In [7]:
input = tot_0[rel_offset:]
target = bt_0[:-rel_offset]

# input = input[14*44100:27*44100]

## Define data set

In [8]:
sample_size = 1024
n_samples = 10240 * 4

In [9]:
x, y = create_training_data_set(n_samples, sample_size, input, target)

In [10]:
x, y = map(apply_fourier_on_dataset, (x, y))

In [12]:
input_shape = x[0].shape[0]
output_shape = x[1].shape[0]

In [13]:
# https://blog.keras.io/building-autoencoders-in-keras.html

from keras.layers import Dense, Dropout, PReLU, Input
from keras.models import Model
from keras.optimizers import Adam

from keras.regularizers import l2
from keras.models import load_model

In [141]:
regularizer_config = {} #'kernel_regularizer': l2(l=0),
                      # 'bias_regularizer': l2(l=0)}
model_name = "vpk_ae.h5"

In [142]:
create_model = True
if create_model:
    encoding_dim = 50

    # Encoder
    input_ = Input((input_shape,))
    _ = input_
    _ = Dense(2000, **regularizer_config)(_)
    _ = PReLU()(_)
    _ = Dense(2000, activity_regularizer=l2(1e-5))(_)
    _ = PReLU()(_)
    # _ = Dropout(0.01)(_)
    # _ = Dense(4000, **regularizer_config)(input_)
    # _ = PReLU()(_)
    # _ = Dropout(0.01)(_)
    # _ = Dense(4000, **regularizer_config)(_)
    # _ = PReLU()(_)
    # _ = Dropout(0.01)(_)
    encoded = Dense(encoding_dim)(_) # , activity_regularizer=l2(1e-4))(_)

    # Decoder
    _ = encoded
    _ = PReLU()(_)
    _ = Dense(2000, activity_regularizer=l2(1e-5))(_)
    _ = PReLU()(_)
    _ = Dense(2000, **regularizer_config)(_)
    _ = PReLU()(_)
    # _ = PReLU()(_)
    # _ = Dropout(0.01)(_)
    # _ = Dense(4000, **regularizer_config)(_)
    # _ = Dropout(0.01)(_)
    # _ = PReLU()(_)
    # _ = Dense(4000, **regularizer_config)(_)
    # _ = Dropout(0.01)(_)
    # _ = PReLU()(_)
    decoded = Dense(output_shape)(_)
    model = Model(input_, decoded)

    model.compile(Adam(lr=0.001), 'mse')
else:
    model = load_model(model_name)

    # create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = encoded_input
# decoder_layer = model.layers[-10](encoded_input)
# decoder_layer = model.layers[-9](decoder_layer)
# decoder_layer = model.layers[-8](decoder_layer)
# decoder_layer = model.layers[-7](decoder_layer)
decoder_layer = model.layers[-6](decoder_layer)
decoder_layer = model.layers[-5](decoder_layer)
decoder_layer = model.layers[-4](decoder_layer)
decoder_layer = model.layers[-3](decoder_layer)
decoder_layer = model.layers[-2](decoder_layer)
decoder_layer = model.layers[-1](decoder_layer)
# create the decoder model
decoder = Model(encoded_input, decoder_layer)

In [143]:
# play_array(model_predict(model, input, sample_size, rfft, irfft), do_wait_done=False, range_secs=[20, 24])

In [144]:
# prediction = model.predict(apply_fourier_on_input(input, sample_size))
# prediction = parallel_apply_along_axis(irfft, 0, prediction).reshape(-1)

In [145]:
# play_array(prediction)
max_duration = len(input) // 44100
prev_loss = np.Infinity
model.save(model_name)
prev_losses = None

In [168]:
n_to_sample = len(x) // 5

In [169]:
import keras.backend as K

In [170]:
K.get_value(model.optimizer.lr)

9.483979e-06

In [ ]:
while True:
    print(f"Current LR: {K.get_value(model.optimizer.lr)}")
    start = dt.datetime.now()

    losses = 1 / (np.sum((model.predict(x) - x)**2, axis=1) + 1e-5)
    
    if prev_losses is not None:
        loss_diff = (losses - prev_losses)**2
        loss_diff_sig = loss_diff / np.mean(loss_diff)
        p_choose = loss_diff_sig / np.sum(loss_diff_sig)
    else:
        print("Initializing p_choose...")
        p_choose = np.ones(shape=losses.shape)
        p_choose /= np.sum(p_choose)
        
    prev_losses = losses
    ix = np.random.choice(len(x), p=p_choose, size=n_to_sample, replace=False)

    history = model.fit(x[ix], x[ix], epochs=4, verbose=1, batch_size=64) # shuffle=True, steps_per_epoch=64)

    # Save model if improvement
    if history.history['loss'][0] < prev_loss * 1.5:
        model.save(model_name)
        model.optimizer.lr = model.optimizer.lr * (1.01 if history.history['loss'][0] < prev_loss else 0.99)
        prev_loss = history.history['loss'][0]
    else:
        print("Failed to find improvement. Loading previous model...")
        model = load_model(model_name)
        model.optimizer.lr = 0.9*model.optimizer.lr
        
    
    
    # Make prediction
    prediction = model_predict(model, input, sample_size, rfft, irfft)
    time_taken = (dt.datetime.now() - start).seconds
    # Play prediction
    play_array(prediction, range_secs=[0, 0 + min(time_taken, max_duration - 1)], do_wait_done=False)

Current LR: 9.483978828939144e-06
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 221.2815
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 220.0374
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 218.9861
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 218.0646
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 9.57881820795592e-06
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 217.7569
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 216.5159
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 215.5463
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 216.0636
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 9.674606189946644e-06
Epoch 1/4
8192/8192 [==============================] - 25

Current LR: 1.068357869371539e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 206.7166
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 205.0557
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 204.0884
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 203.3862
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.057674307958223e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 204.1385
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 202.9445
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 201.8312
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 201.8577
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.0682510037440807e-05
Epoch 1/4
8192/8192 [==============================] - 

Current LR: 1.1334030205034651e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 198.2586
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 197.1335
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 196.3464
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 195.6013
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.1447370525274891e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 201.6978
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 201.8796
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 199.7850
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 197.7267
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.1332896974636242e-05
Epoch 1/4
8192/8192 [==============================] 

Current LR: 1.2266993508092128e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 194.2875
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 193.3907
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 192.4400
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 191.5914
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.2389663424983155e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 194.6218
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 193.1858
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 192.7203
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 191.8489
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.2265766599739436e-05
Epoch 1/4
8192/8192 [==============================] 

Current LR: 1.2756148862536065e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 190.7898
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 189.7930
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 189.0254
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 188.2924
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.2883710041933227e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 191.8838
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 190.7088
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 189.7537
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 189.0272
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.2754872841469478e-05
Epoch 1/4
8192/8192 [==============================] 

...

Current LR: 1.841977791627869e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 165.6490
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 164.9321
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 164.2317
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 163.6428
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.8235579773318022e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 164.7291
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 163.9296
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 163.3715
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 162.8604
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.8417935280012898e-05
Epoch 1/4
8192/8192 [==============================] -

Current LR: 1.8038786947727203e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 164.0176
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 163.0417
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 162.3880
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 161.9524
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.8219174307887442e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 163.2956
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 162.5760
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 162.0209
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 161.5810
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.8401366105536e-05
Epoch 1/4
8192/8192 [==============================] - 2

Current LR: 1.838665411924012e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 162.3313
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 161.5539
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 161.0002
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 160.5658
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.8570521206129342e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 163.4458
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 162.5891
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 162.1038
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 161.4707
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.838481693994254e-05
Epoch 1/4
8192/8192 [==============================] - 

Current LR: 1.9119835997116752e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 161.7674
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 160.9561
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 160.4797
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 159.8879
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.9311033611302264e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 161.8020
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 161.0145
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 160.3738
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 160.0357
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.9117924239253625e-05
Epoch 1/4
8192/8192 [==============================] 

Current LR: 1.9488550606183708e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 160.5579
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 159.8808
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 159.3356
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 158.8973
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.9683435311890207e-05
Epoch 1/4
8192/8192 [==============================] - 25s 3ms/step - loss: 160.6682
Epoch 2/4
8192/8192 [==============================] - 25s 3ms/step - loss: 159.8737
Epoch 3/4
8192/8192 [==============================] - 25s 3ms/step - loss: 159.3952
Epoch 4/4
8192/8192 [==============================] - 25s 3ms/step - loss: 158.8413
FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...
Current LR: 1.9486600649543107e-05
Epoch 1/4
8192/8192 [==============================] 

In [62]:
history.history['loss'][0]

3291.7697265625

In [ ]:
# play_array(model_predict(model, input, sample_size), range_secs=[130, 130 + min(time_taken, 250)], do_wait_done=False)

In [ ]:
# play_array(model_predict(model, input, sample_size), range_secs=[20, 24])  

In [ ]:
# play_array(x[0], range_secs=[20,24])
# play_array(target, range_secs=[20,24])

## Test on different file

In [154]:
test = read_file("../data/Wasmachine_in_a_mineur_v5.wav")
prediction = model_predict(model, test, sample_size, rfft, irfft)
play_array(prediction, range_secs=[0, 0 + min(time_taken, 60)], do_wait_done=False)

FFT transforming...
Predicting and transforming...
Applying x-fade and mixing...


In [ ]:
encoder = Model(input_, encoded)
pred = encoder.predict(apply_fourier_on_input(input, sample_size))

In [ ]:
import pandas as pd

In [ ]:
pd.Series(pred.reshape(-1)).hist(bins=51, range=[-2, 2])

In [ ]:
# percentiles = np.percentile(pred, 0, axis=1)
# pred = np.array([np.where(x > p, x, 0) for x, p in zip(pred, percentiles)])

In [ ]:
pred = decoder_predict(decoder, pred, irfft)
pred -= np.mean(pred)

In [ ]:
play_array(pred, range_secs=[10,20])

In [ ]:
%matplotlib inline

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(np.arange(0, len(pred)), pred[:, 0], pred[:, 1])

In [ ]:
# _ = model.predict(apply_fourier_on_input(input, sample_size))
# play_array(parallel_apply_along_axis(irfft, 0, _).reshape(-1), range_secs=[0,60])

In [ ]:
decoder.predict(x_enc).shape

In [ ]:
x_enc = np.zeros(shape=(10, encoding_dim))

# x_enc = np.array([np.sin(np.arange(0, 2000) / 40) for x in range(0, 10)])

pred = decoder_predict(decoder, x_enc, irfft)

pred -= np.mean(pred)
pred /= (np.percentile(pred, 95) - np.percentile(pred, 5)) * 2

In [ ]:
plt.plot(pred)

In [ ]:
play_array(pred)